In [1]:
import pandas as pd
import numpy as np
from faker import Faker
import random
import datetime
import boto3
import psycopg2
import configparser
from sqlalchemy import create_engine
import pyodbc

In [2]:
config = configparser.ConfigParser()
config.read('escproy.cfg')

['escproy.cfg']

In [3]:
RDS_HOST = 'proyecto.cjsvaexfxmdn.us-east-1.rds.amazonaws.com'
#RDS_HOST=config.get('DW', 'RDS_HOST')
#RDS_HOST
print(RDS_HOST)

proyecto.cjsvaexfxmdn.us-east-1.rds.amazonaws.com


In [4]:
postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""  

##### Creamos Dimensión de Clientes

In [5]:
sql_query = 'SELECT * FROM ventas;'
clientes1 = pd.read_sql(sql_query, postgres_driver)
clientes1.head()

,index,Id. de la fila,Id. del pedido,Fecha del pedido,Fecha de envío,Forma de envío,Id. del cliente,Nombre del cliente,Segmento,Ciudad,...,País/Región,Región,Id. del producto,Categoría,Subcategoría,Nombre del producto,Ventas,Cantidad,Descuento,Ganancia
0,0,1682,MX-2021-111899,2021-04-02,2021-04-06,Estándar,AM-11365,Araceli Manzanares,Empresa,Cruzeiro do Sul,...,Brasil,Sur,MAT-AR-10004857,Material de oficina,Arte,"Boston Rotuladores, Tamaños variados",591.0,3,0.0,206.4
1,1,5919,MX-2021-113922,2021-10-08,2021-10-11,Rápido,BS-12130,Benjamín Saavedra,Cliente,Rio Branco,...,Brasil,Sur,MOB-LI-10000647,Mobiliario,Librerías,"Dania Conjunto de estantes, Metal",2250.8,2,0.0,922.8
2,2,5920,MX-2021-113922,2021-10-08,2021-10-11,Rápido,BS-12130,Benjamín Saavedra,Cliente,Rio Branco,...,Brasil,Sur,MAT-CA-10002956,Material de oficina,Carpetas,"Avery Anillas, Transparente",129.0,5,0.0,11.0
3,3,9013,MX-2019-163888,2019-06-14,2019-06-18,Estándar,AT-10090,Alejandro Trejo,Cliente,Rio Branco,...,Brasil,Sur,MOB-MO-10002100,Mobiliario,Mobiliario,"Tenex Bandeja apiladora, Negro",504.6,3,0.0,95.4
4,4,9014,MX-2019-163888,2019-06-14,2019-06-18,Estándar,AT-10090,Alejandro Trejo,Cliente,Rio Branco,...,Brasil,Sur,MOB-LI-10002747,Mobiliario,Librerías,"Safco Vitrina, Convencional",3376.8,3,0.0,641.4


In [6]:
dfClientes=clientes1.drop(['index','Id. de la fila','Id. del pedido','Fecha del pedido',
       'Fecha de envío', 'Forma de envío','Id. del producto', 'Categoría', 'Subcategoría',
       'Nombre del producto', 'Ventas', 'Cantidad', 'Descuento', 'Ganancia'],axis=1)
dfClientes

,Id. del cliente,Nombre del cliente,Segmento,Ciudad,Estado,País/Región,Región
0,AM-11365,Araceli Manzanares,Empresa,Cruzeiro do Sul,Acre,Brasil,Sur
1,BS-12130,Benjamín Saavedra,Cliente,Rio Branco,Acre,Brasil,Sur
2,BS-12130,Benjamín Saavedra,Cliente,Rio Branco,Acre,Brasil,Sur
3,AT-10090,Alejandro Trejo,Cliente,Rio Branco,Acre,Brasil,Sur
4,AT-10090,Alejandro Trejo,Cliente,Rio Branco,Acre,Brasil,Sur
...,...,...,...,...,...,...,...
10249,VM-21460,Valentina Macías,Pequeña empresa,Cabimas,Zulia,Venezuela,Sur
10250,VM-21460,Valentina Macías,Pequeña empresa,Cabimas,Zulia,Venezuela,Sur
10251,VM-21460,Valentina Macías,Pequeña empresa,Cabimas,Zulia,Venezuela,Sur
10252,VM-21460,Valentina Macías,Pequeña empresa,Cabimas,Zulia,Venezuela,Sur


In [7]:
sql_query = 'SELECT * FROM personas;'
dfpersonas = pd.read_sql(sql_query, postgres_driver)
dfpersonas.head()

,index,id gerente,Región,Gerente regional
0,0,1001,Caribe,Eva Lara
1,1,1002,Centro,Silvia Pérez
2,2,1003,Norte,Augusto Olivares
3,3,1004,Sur,Gabriela Bustamante


In [8]:
dim_Clientes = dfClientes.merge(dfpersonas, on='Región', how='inner')
dim_Clientes.head()

,Id. del cliente,Nombre del cliente,Segmento,Ciudad,Estado,País/Región,Región,index,id gerente,Gerente regional
0,AM-11365,Araceli Manzanares,Empresa,Cruzeiro do Sul,Acre,Brasil,Sur,3,1004,Gabriela Bustamante
1,BS-12130,Benjamín Saavedra,Cliente,Rio Branco,Acre,Brasil,Sur,3,1004,Gabriela Bustamante
2,BS-12130,Benjamín Saavedra,Cliente,Rio Branco,Acre,Brasil,Sur,3,1004,Gabriela Bustamante
3,AT-10090,Alejandro Trejo,Cliente,Rio Branco,Acre,Brasil,Sur,3,1004,Gabriela Bustamante
4,AT-10090,Alejandro Trejo,Cliente,Rio Branco,Acre,Brasil,Sur,3,1004,Gabriela Bustamante


In [78]:
dim_Clientes=dim_Clientes.drop(['index'],axis=1)
dim_Clientes=dim_Clientes.drop_duplicates('Id. del cliente')
dim_Clientes.head()

,Id. del cliente,Nombre del cliente,Segmento,Ciudad,Estado,País/Región,Región,id gerente,Gerente regional
0,AM-11365,Araceli Manzanares,Empresa,Cruzeiro do Sul,Acre,Brasil,Sur,1004,Gabriela Bustamante
1,BS-12130,Benjamín Saavedra,Cliente,Rio Branco,Acre,Brasil,Sur,1004,Gabriela Bustamante
3,AT-10090,Alejandro Trejo,Cliente,Rio Branco,Acre,Brasil,Sur,1004,Gabriela Bustamante
5,AJ-10840,Amador Jasón,Cliente,Cruzeiro do Sul,Acre,Brasil,Sur,1004,Gabriela Bustamante
6,LG-17920,Leonor Gómez,Cliente,Penedo,Alagoas,Brasil,Sur,1004,Gabriela Bustamante


#### Dimension de Forma de envio

In [10]:
sql_query = 'SELECT * FROM ventas;'
Formasenvio = pd.read_sql(sql_query, postgres_driver)
Formasenvio1=Formasenvio.drop(['index', 'Id. de la fila', 'Id. del pedido', 'Fecha del pedido',
       'Fecha de envío', 'Id. del cliente',
       'Nombre del cliente', 'Segmento', 'Ciudad', 'Estado', 'País/Región',
       'Región', 'Id. del producto', 'Categoría', 'Subcategoría',
       'Nombre del producto', 'Ventas', 'Cantidad', 'Descuento', 'Ganancia'],axis=1)
Formasenvio2=Formasenvio1.drop_duplicates('Forma de envío')
Formasenvio2

,Forma de envío
0,Estándar
1,Rápido
5,Urgente
19,Mismo día


In [64]:
FormasEnvio3=pd.DataFrame(Formasenvio2)
FormasEnvio3['id forma envio']=[101,102,103,104]
dimEnvio=FormasEnvio3.reindex(columns=['id forma envio','Forma de envío'])
dimEnvio

,id forma envio,Forma de envío
0,101,Estándar
1,102,Rápido
5,103,Urgente
19,104,Mismo día


#### Dimension de Producto

In [12]:
sql_query = 'SELECT * FROM ventas;'
productos = pd.read_sql(sql_query, postgres_driver)
productos1=productos.drop(['index', 'Id. de la fila', 'Id. del pedido', 'Fecha del pedido',
       'Fecha de envío', 'Id. del cliente',
       'Nombre del cliente', 'Segmento', 'Ciudad', 'Estado', 'País/Región',
       'Región', 'Forma de envío','Ventas', 'Cantidad', 'Descuento', 'Ganancia'],axis=1)
DimProducto=productos1.drop_duplicates('Id. del producto')
DimProducto.head()

,Id. del producto,Categoría,Subcategoría,Nombre del producto
0,MAT-AR-10004857,Material de oficina,Arte,"Boston Rotuladores, Tamaños variados"
1,MOB-LI-10000647,Mobiliario,Librerías,"Dania Conjunto de estantes, Metal"
2,MAT-CA-10002956,Material de oficina,Carpetas,"Avery Anillas, Transparente"
3,MOB-MO-10002100,Mobiliario,Mobiliario,"Tenex Bandeja apiladora, Negro"
4,MOB-LI-10002747,Mobiliario,Librerías,"Safco Vitrina, Convencional"


##### Dimensión Calendario

In [13]:
sql_query = 'SELECT * FROM ventas;'
calendario = pd.read_sql(sql_query, postgres_driver)
Dim_calendario=calendario.drop(['index', 'Id. de la fila', 'Id. del pedido', 'Forma de envío','Fecha de envío', 'Id. del cliente',
       'Nombre del cliente', 'Segmento', 'Ciudad', 'Estado', 'País/Región',
       'Región', 'Id. del producto', 'Categoría', 'Subcategoría',
       'Nombre del producto', 'Ventas', 'Cantidad', 'Descuento', 'Ganancia'],axis=1)
Dim_calendario.head()

,Fecha del pedido
0,2021-04-02
1,2021-10-08
2,2021-10-08
3,2019-06-14
4,2019-06-14


In [14]:
Dim_calendario['day'] = pd.DatetimeIndex(Dim_calendario['Fecha del pedido']).day
Dim_calendario['month'] = pd.DatetimeIndex(Dim_calendario['Fecha del pedido']).month
Dim_calendario['year'] = pd.DatetimeIndex(Dim_calendario['Fecha del pedido']).year
Dim_calendario['quarter'] = pd.DatetimeIndex(Dim_calendario['Fecha del pedido']).quarter
Dim_calendario.head()

,Fecha del pedido,day,month,year,quarter
0,2021-04-02,2,4,2021,2
1,2021-10-08,8,10,2021,4
2,2021-10-08,8,10,2021,4
3,2019-06-14,14,6,2019,2
4,2019-06-14,14,6,2019,2


In [15]:
Dim_calendario['id_fecha'] = Dim_calendario['year'].astype(str) + Dim_calendario['month'].astype(str)
Dim_calendario['id_fecha'] = Dim_calendario['id_fecha'].astype(str) + Dim_calendario['day'].astype(str)
Dim_calendario.drop_duplicates(inplace=True)
Dim_calendario.head()

,Fecha del pedido,day,month,year,quarter,id_fecha
0,2021-04-02,2,4,2021,2,202142
1,2021-10-08,8,10,2021,4,2021108
3,2019-06-14,14,6,2019,2,2019614
5,2018-07-20,20,7,2018,3,2018720
6,2021-05-16,16,5,2021,2,2021516


In [16]:
Dim_calendario

,Fecha del pedido,day,month,year,quarter,id_fecha
0,2021-04-02,2,4,2021,2,202142
1,2021-10-08,8,10,2021,4,2021108
3,2019-06-14,14,6,2019,2,2019614
5,2018-07-20,20,7,2018,3,2018720
6,2021-05-16,16,5,2021,2,2021516
...,...,...,...,...,...,...
10105,2018-04-26,26,4,2018,2,2018426
10106,2021-02-21,21,2,2021,1,2021221
10111,2021-07-06,6,7,2021,3,202176
10127,2020-08-11,11,8,2020,3,2020811


#### Tabla de Hechos

In [17]:
sql_query = 'SELECT * FROM ventas;'
TabHechos = pd.read_sql(sql_query, postgres_driver)
TabHechos1=TabHechos.drop(['index', 'Id. de la fila',
       'Nombre del cliente', 'Segmento', 'Ciudad', 'Estado', 'País/Región',
       'Región', 'Categoría', 'Subcategoría',
       'Nombre del producto'],axis=1)
TabHechos1.head()

,Id. del pedido,Fecha del pedido,Fecha de envío,Forma de envío,Id. del cliente,Id. del producto,Ventas,Cantidad,Descuento,Ganancia
0,MX-2021-111899,2021-04-02,2021-04-06,Estándar,AM-11365,MAT-AR-10004857,591.0,3,0.0,206.4
1,MX-2021-113922,2021-10-08,2021-10-11,Rápido,BS-12130,MOB-LI-10000647,2250.8,2,0.0,922.8
2,MX-2021-113922,2021-10-08,2021-10-11,Rápido,BS-12130,MAT-CA-10002956,129.0,5,0.0,11.0
3,MX-2019-163888,2019-06-14,2019-06-18,Estándar,AT-10090,MOB-MO-10002100,504.6,3,0.0,95.4
4,MX-2019-163888,2019-06-14,2019-06-18,Estándar,AT-10090,MOB-LI-10002747,3376.8,3,0.0,641.4


In [18]:
TabHechos2=TabHechos1.merge(dimFormasEnvio, on='Forma de envío', how='inner')
dfFactTable=TabHechos2.drop(['Forma de envío'],axis=1)
dfFactTable

,Id. del pedido,Fecha del pedido,Fecha de envío,Id. del cliente,Id. del producto,Ventas,Cantidad,Descuento,Ganancia,id forma envio
0,MX-2021-111899,2021-04-02,2021-04-06,AM-11365,MAT-AR-10004857,591.00,3,0.0,206.40,101
1,MX-2019-163888,2019-06-14,2019-06-18,AT-10090,MOB-MO-10002100,504.60,3,0.0,95.40,101
2,MX-2019-163888,2019-06-14,2019-06-18,AT-10090,MOB-LI-10002747,3376.80,3,0.0,641.40,101
3,MX-2020-133459,2020-09-05,2020-09-09,CV-12475,MAT-SO-10003712,540.00,3,0.0,37.80,101
4,MX-2020-166443,2020-08-21,2020-08-25,WV-21820,MAT-PA-10000990,1248.00,6,0.0,261.60,101
...,...,...,...,...,...,...,...,...,...,...
10249,MX-2021-155649,2021-10-21,2021-10-21,JU-16765,TEC-MÁ-10002952,1136.80,2,0.0,443.20,104
10250,MX-2018-157210,2018-12-23,2018-12-23,YP-21895,TEC-AC-10003337,614.40,3,0.0,282.60,104
10251,MX-2020-116624,2020-11-09,2020-11-09,AB-10180,MAT-SO-10001166,342.40,1,0.0,27.20,104
10252,MX-2020-116624,2020-11-09,2020-11-09,AB-10180,MAT-ET-10003927,148.80,2,0.0,22.00,104


In [19]:
dfFactTable['year'] = pd.DatetimeIndex(dfFactTable['Fecha del pedido']).year
dfFactTable['month'] = pd.DatetimeIndex(dfFactTable['Fecha del pedido']).month
dfFactTable['day'] = pd.DatetimeIndex(dfFactTable['Fecha del pedido']).day
dfFactTable['id_fecha'] = dfFactTable['year'].astype(str) + dfFactTable['month'].astype(str)
dfFactTable['id_fecha'] = dfFactTable['id_fecha'].astype(str) + dfFactTable['day'].astype(str)
factTable_transacciones = dfFactTable.drop(['year', 'month', 'day'], axis=1)
factTable_transacciones.head()

,Id. del pedido,Fecha del pedido,Fecha de envío,Id. del cliente,Id. del producto,Ventas,Cantidad,Descuento,Ganancia,id forma envio,id_fecha
0,MX-2021-111899,2021-04-02,2021-04-06,AM-11365,MAT-AR-10004857,591.0,3,0.0,206.4,101,202142
1,MX-2019-163888,2019-06-14,2019-06-18,AT-10090,MOB-MO-10002100,504.6,3,0.0,95.4,101,2019614
2,MX-2019-163888,2019-06-14,2019-06-18,AT-10090,MOB-LI-10002747,3376.8,3,0.0,641.4,101,2019614
3,MX-2020-133459,2020-09-05,2020-09-09,CV-12475,MAT-SO-10003712,540.0,3,0.0,37.8,101,202095
4,MX-2020-166443,2020-08-21,2020-08-25,WV-21820,MAT-PA-10000990,1248.0,6,0.0,261.6,101,2020821


##### Creación de Instancias en AWS para DW

In [20]:
config = configparser.ConfigParser()
config.read('escproy.cfg')

['escproy.cfg']

In [21]:
aws_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                    aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                    region_name='us-east-1')
print(aws_conn)

In [22]:
rdsInstanceIds = []

response = aws_conn.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['banco-db', 'dw-db', 'dw-proy', 'proyecto']


In [23]:
rdsIdentifier = 'dw-proy'

In [24]:
try:
    response = aws_conn.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS_POST', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="postgres",
            MasterUsername=config.get('RDS_POST', 'DB_USER'),
            MasterUserPassword=config.get('RDS_POST', 'DB_PASSWORD'),
            Port=int(config.get('RDS_POST', 'DB_PORT')),
            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
            PubliclyAccessible=True
        )
    print(response)
except aws_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.")

La Instancia de Base de Datos ya Existe.


In [25]:
try:
     instances = aws_conn.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_DW_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_DW_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)

dw-proy.cjsvaexfxmdn.us-east-1.rds.amazonaws.com


In [26]:
import create_dw_p

try:
    db_conn = psycopg2.connect(
        database=config.get('RDS_POST', 'DB_NAME'), 
        user=config.get('RDS_POST', 'DB_USER'),
        password=config.get('RDS_POST', 'DB_PASSWORD'), 
        host=RDS_DW_HOST,
        port=config.get('RDS_POST', 'DB_PORT')
    )

    cursor = db_conn.cursor()
    cursor.execute(create_dw_p.CREATE_DW)
    db_conn.commit()
    print("Base de Datos Creada Exitosamente")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex) 

Base de Datos Creada Exitosamente


##### Insertamos Data

In [80]:
postgres_driver = f"""postgresql://{config.get('RDS_POST', 'DB_USER')}:{config.get('RDS_POST', 'DB_PASSWORD')}@{RDS_DW_HOST}:{config.get('RDS_POST', 'DB_PORT')}/{config.get('RDS_POST', 'DB_NAME')}"""  
dim_Clientes.to_sql('dimclientes', postgres_driver, if_exists='replace')
postgres_driver = psycopg2.connect(postgres_driver)

In [69]:
postgres_driver = f"""postgresql://{config.get('RDS_POST', 'DB_USER')}:{config.get('RDS_POST', 'DB_PASSWORD')}@{RDS_DW_HOST}:{config.get('RDS_POST', 'DB_PORT')}/{config.get('RDS_POST', 'DB_NAME')}"""
DimProducto.to_sql('dimproducto', postgres_driver, if_exists='replace')
postgres_driver = psycopg2.connect(postgres_driver)

In [70]:
postgres_driver = f"""postgresql://{config.get('RDS_POST', 'DB_USER')}:{config.get('RDS_POST', 'DB_PASSWORD')}@{RDS_DW_HOST}:{config.get('RDS_POST', 'DB_PORT')}/{config.get('RDS_POST', 'DB_NAME')}"""
dimEnvio.to_sql('dimenvio', postgres_driver, if_exists='replace')
postgres_driver = psycopg2.connect(postgres_driver)

In [71]:
postgres_driver = f"""postgresql://{config.get('RDS_POST', 'DB_USER')}:{config.get('RDS_POST', 'DB_PASSWORD')}@{RDS_DW_HOST}:{config.get('RDS_POST', 'DB_PORT')}/{config.get('RDS_POST', 'DB_NAME')}"""
Dim_calendario.to_sql('dimcalendario', postgres_driver, if_exists='replace')
postgres_driver = psycopg2.connect(postgres_driver)

In [72]:
postgres_driver = f"""postgresql://{config.get('RDS_POST', 'DB_USER')}:{config.get('RDS_POST', 'DB_PASSWORD')}@{RDS_DW_HOST}:{config.get('RDS_POST', 'DB_PORT')}/{config.get('RDS_POST', 'DB_NAME')}"""
factTable_transacciones.to_sql('fact_table', postgres_driver, if_exists='replace')
postgres_driver = psycopg2.connect(postgres_driver)